# Airbus Ship Detection

EE 435: Deep Learning Foundations from Scratch \
Northwestern Univertiy Winter 2020

## Memebers
* Blaine Rothrock (`BRY4768`)
* Ilan Ponsky (`IPW1530`)
* Will Dong (`WDG4518`)

## Overview

Based on the kaggle challenge [Airbus Ship Detection Challenge](https://www.kaggle.com/c/airbus-ship-detection/overview). This challenge uses satellite images to detect if a image has ships and draw a box (mask) around the ship. This challenge contains ~40GB of data and the training size is a little out of scope for this project, so we approaching it with two goals:
1. Training a binary classifier to classify images that contain ship(s)
2. Explore the state-of-the-art approach to this model, U-Net
    - Goal is to follow the baseline notebook submission and understand the architechure. 

## Step 1 Understanding the data and problem space

// TODO

## Binary Classification Results

 - followed [this](https://www.kaggle.com/inversion/run-length-decoding-quick-start) Kaggle note book for getting started with the 